In [15]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import logistic_regression as lr
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [279]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [280]:
from helpers import *
x, mean_x, std_x = standardize(tX)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [5]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '../data/forcast.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [281]:
y= ((y+1)/2).reshape(len(y),1)

In [120]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [249]:
# Useful starting lines
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(t):
    """apply sigmoid function on t."""
    return 1/(1+np.exp(-t))

def calculate_loss(y, tx, w):
    return -(y.transpose().dot(np.log(sigmoid(tx.dot(w)))) + (1-y.transpose()).dot(np.log(1-sigmoid(tx.dot(w)))))

def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.transpose().dot(sigmoid(tx.dot(w))-y)

def learning_by_gradient_descent(y, tx, w, alpha):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    grad = calculate_gradient(y, tx, w)
    w = w - alpha*grad
    loss = calculate_loss(y, tx, w)
    return loss, w

from helpers import standardize

def logistic_regression_gradient_descent(y, tx, alpha = 0.001, max_iter =1000, verbatim = False):
    # init parameters
    threshold = 1e-8
    losses = []
    
    w = np.zeros((tx.shape[1], 1))

    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, alpha)
        # log info
        if verbatim and iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        #print loss
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return w, losses[-1]

In [253]:
def cross_validation(y, x, k_indices, k, lambda_, degree, max_iter =1000):
    """return the loss of ridge regression."""
    
    x_test = x[k_indices[k]]
    y_test = y[k_indices[k]]
    
    #print len(k_indices[[l for l in range(len(k_indices)) if l!=k]])
    id = np.array(k_indices[[l for l in range(len(k_indices)) if l!=k]]).reshape((len(k_indices)-1)*len(k_indices[k]),1)
    id = id.reshape((len(id),))

    x_train = x[id]
    y_train = y[id]
    
    w, loss_tr = logistic_regression_gradient_descent(y_train, x_train, lambda_)
    
    pred =np.array([0.0 if t < 0.5 else 1.0 for t in  np.log(sigmoid(x_test.dot(w)))])
    rate = float(sum(pred==y_test)[0])/len(y_test)
    print rate
    loss_te = calculate_loss(y_test, x_test, w)
    return loss_tr, loss_te

In [256]:
def cross_validation(y, x, k_indices, k, lambda_, degree, max_iter =1000):
    """return the loss of ridge regression."""
    
    x_test = x[k_indices[k]]
    y_test = y[k_indices[k]]
    
    #print len(k_indices[[l for l in range(len(k_indices)) if l!=k]])
    id = np.array(k_indices[[l for l in range(len(k_indices)) if l!=k]]).reshape((len(k_indices)-1)*len(k_indices[k]),1)
    id = id.reshape((len(id),))

    x_train = x[id]
    y_train = y[id]
    
    w, loss_tr = logistic_regression_gradient_descent(y_train, x_train, lambda_)
    
    pred =np.array([0.0 if t < 0.5 else 1.0 for t in  np.log(sigmoid(x_test.dot(w)))])
    rate = float(sum(pred==y_test)[0])/len(y_test)
    print rate
    loss_te = calculate_loss(y_test, x_test, w)
    return loss_tr, loss_te

def cross_validation_demo(max_iter =100):
    seed = 1
    degree = 7
    k_fold = 4
    lambdas = np.logspace(-7, -5, 10)
    
    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    vrmse_tr = []
    vrmse_te = []
    for lambda_ in lambdas[4:5]:
        rmse_tr1 = []
        rmse_te1 = []
        for k in range(k_fold):
            rmse_tR, rmse_tE =cross_validation(y, x, k_indices, k, lambda_, degree, max_iter =1000)
            print rmse_tR[0,0], rmse_tE[0,0]
            rmse_tr1.append(rmse_tR)
            rmse_te1.append(rmse_tE)
        rmse_tr.append(np.mean(rmse_tr1))
        rmse_te.append(np.mean(rmse_te1))
        vrmse_tr.append(np.var(rmse_tr1))
        vrmse_te.append(np.var(rmse_te1))
    #cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    return rmse_tr, rmse_te, vrmse_tr, vrmse_te

In [257]:
import logistic_regression as lr
from time import time
st = time()
result = cross_validation_demo()
print time() - st

0.660816
93517.9518483 31187.6437963
0.654976
93481.492709 31230.1633227


KeyboardInterrupt: 

In [247]:
w, loss = logistic_regression_gradient_descent(y, x, np.logspace(-7, -5, 10)[4], max_iter =10000)

Current iteration=0, the loss=[[ 166069.54342234]]
Current iteration=100, the loss=[[ 129939.87315837]]
Current iteration=200, the loss=[[ 127516.58749782]]
Current iteration=300, the loss=[[ 126313.33721582]]
Current iteration=400, the loss=[[ 125624.93088041]]
Current iteration=500, the loss=[[ 125205.14622196]]
Current iteration=600, the loss=[[ 124937.9615187]]
Current iteration=700, the loss=[[ 124762.55578739]]
Current iteration=800, the loss=[[ 124644.69828371]]
Current iteration=900, the loss=[[ 124564.07626362]]
Current iteration=1000, the loss=[[ 124508.1368841]]
Current iteration=1100, the loss=[[ 124468.87337979]]
Current iteration=1200, the loss=[[ 124441.04928101]]
Current iteration=1300, the loss=[[ 124421.17058612]]
Current iteration=1400, the loss=[[ 124406.86774369]]
Current iteration=1500, the loss=[[ 124396.51217571]]
Current iteration=1600, the loss=[[ 124388.97205345]]
Current iteration=1700, the loss=[[ 124383.45335584]]
Current iteration=1800, the loss=[[ 124379

In [259]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [261]:
from helpers import *
x, mean_x, std_x = standardize(tX_test)

In [267]:
tX_test = tx = np.c_[np.ones((tX_test.shape[0], 1)), tX_test]

In [268]:
OUTPUT_PATH = '../data/forcast.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [192]:
# penalisation

In [289]:
def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    txw = tx.dot(w)
    S= np.diag(np.multiply(sigmoid(txw),1-sigmoid(txw))[:,0])
    return tx.transpose().dot(S.dot(tx))

def calculate_penalized_loss(y, tx, w, lambda_):
    return calculate_loss(y, tx, w) + lambda_*w.transpose().dot(w)

def calculate_penalized_gradient(y, tx, w, lambda_):
    """compute the gradient of loss."""
    return calculate_gradient(y, tx, w) + 2*lambda_*w

def calculate_penalized_hessian(y, tx, w, lambda_):
    return calculate_hessian(y, tx, w) + 2*lambda_*np.identity(len(w))

In [290]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    loss = calculate_penalized_loss(y, tx, w, lambda_)
    grad = calculate_penalized_gradient(y, tx, w, lambda_)
    #H = calculate_penalized_hessian(y, tx, w, lambda_)
    return loss, grad#, H

In [291]:
def learning_by_penalized_gradient(y, tx, w, alpha, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """    
    loss, grad = penalized_logistic_regression(y, tx, w, lambda_)
    w = w - alpha*grad
    loss = calculate_penalized_loss(y, tx, w, lambda_)
    return loss, w

In [292]:
def logistic_regression_penalized_gradient_descent_demo(y, x, alpha, lambda_= 0.1, max_iter = 10000):
    # init parameters
    #max_iter = 10000
    #alpha = 0.01
    #lambda_ = 0.1
    threshold = 1e-8
    losses = []

    w = np.zeros((x.shape[1], 1))
    calculate_penalized_loss(y, tx, w, alpha)
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, x, w, alpha, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return w, losses[-1]
    

In [293]:
def cross_validation(y, x, k_indices, k, alpha, degree):
    """return the loss of ridge regression."""
    
    x_test = x[k_indices[k]]
    y_test = y[k_indices[k]]
    
    #print len(k_indices[[l for l in range(len(k_indices)) if l!=k]])
    id = np.array(k_indices[[l for l in range(len(k_indices)) if l!=k]]).reshape((len(k_indices)-1)*len(k_indices[k]),1)
    id = id.reshape((len(id),))

    x_train = x[id]
    y_train = y[id]
    
    w, loss_tr = logistic_regression_penalized_gradient_descent_demo(y_train, x_train, alpha)
    
    pred =np.array([0.0 if t < 0.5 else 1.0 for t in  np.log(sigmoid(x_test.dot(w)))])
    loss_te = calculate_loss(y_test, x_test, w)
    return loss_tr, loss_te

In [294]:
def cross_validation_demo(lambda_):
    seed = 1
    degree = 7
    k_fold = 4
    #lambdas = np.logspace(-7, -5, 10)
    alphas = np.logspace(-3, 1, 10)
    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    vrmse_tr = []
    vrmse_te = []
    for alpha in alphas[4:5]:
        rmse_tr1 = []
        rmse_te1 = []
        for k in range(k_fold):
            rmse_tR, rmse_tE =cross_validation(y, x, k_indices, k, alpha, degree)
            print rmse_tR[0,0], rmse_tE[0,0]
            rmse_tr1.append(rmse_tR)
            rmse_te1.append(rmse_tE)
        rmse_tr.append(np.mean(rmse_tr1))
        rmse_te.append(np.mean(rmse_te1))
        vrmse_tr.append(np.var(rmse_tr1))
        vrmse_te.append(np.var(rmse_te1))
    #cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    return rmse_tr, rmse_te, vrmse_tr, vrmse_te

In [295]:
lambda_ = np.logspace(-7, -5, 10)[4]
cross_validation_demo(lambda_)

ValueError: shapes (1,187500) and (568238,1) not aligned: 187500 (dim 1) != 568238 (dim 0)